# Iterables vs. Iterators vs. Generators 

## Let's explore the differences between...
* a container
* an iterable
* an iterator
* a generator
* a generator expression
* a {list, set, dict} comprehension


![alt-text](relationships.png)

## Containers 
* data structures which hold elements
* support membership tests
* live in memory
* typically hold all their values in memory
* e.g., string, list, tuple, set, dict
* an object is a container when it can be asked whether it _contains_ a certain element

In [1]:
1 in [1, 2, 3], 0 in [1, 2, 3]

(True, False)

In [2]:
4 in {4, 5, 6}, 1 in {4, 5, 6}

(True, False)

In [22]:
44 in ('Obama', 'Barack', 44, 2008, 'left')

True

In [3]:
# for dicts, membership checks the keys, not the values
44 in { 43: 'Bush', 44: 'Obama'}, 'Bush' in { 43: 'Bush', 44: 'Obama' }

(True, False)

In [2]:
'J' in 'Steve Jobs', 'Job' in 'Steve Jobs', 'Jobs' not in
'Carlos Jobim'

(True, True, True)

## Iterables
* any object, not necessarily a data structure, that can return an iterator (with the purpose of returning all of its elements)
* the `__iter__()` function returns an iterator
    * ...therefore, any object which has the `__iter__()` method is an iterable
* most containers are also iterable
* many more things are iterable as well (e.g., open files, open sockets, etc.)

In [1]:
mylist = [1, 2, 3]
iter1 = iter(mylist)
print(type(iter1))
iter2 = mylist.__iter__() # iter() maps to __iter__()
print(type(iter2))
next(iter1)

<class 'list_iterator'>
<class 'list_iterator'>


1

In [4]:
iter1.__next__() # next() maps to __next__()

2

In [5]:
next(iter1)

3

In [6]:
next(iter1)

StopIteration: 

In [7]:
next(iter2)

1

In [8]:
type(mylist), type(iter1), type(iter2)

(list, list_iterator, list_iterator)

In [9]:
# a list is iterable, but it is not its own iterator
next(mylist)

TypeError: 'list' object is not an iterator

In [10]:
iter(mylist) is mylist

False

In [ ]:
more = True
itermore = iter(mylist)
while more:
    try:
        i = next(itermore)
    except StopIteration as e:
        more = False

In [12]:
mylistiter = iter(mylist)
mylistiter, id(mylist)

(<list_iterator at 0x105f2a2e8>, 4387766472)

In [16]:
iter(mylistiter) is mylistiter

True

## When we write...
`mylist = [1, 2, 3]
for x in mylist:
    ...`
## ...this is what happens

![alt-text](iterable.png "iterable")

## We can see this by disassembling the Python code...

In [13]:
import dis
mylist = [1, 2, 3]
total = 0
dis.dis('for item in mylist: total += item')

  1           0 SETUP_LOOP              20 (to 22)
              2 LOAD_NAME                0 (mylist)
              4 GET_ITER
        >>    6 FOR_ITER                12 (to 20)
              8 STORE_NAME               1 (item)
             10 LOAD_NAME                2 (total)
             12 LOAD_NAME                1 (item)
             14 INPLACE_ADD
             16 STORE_NAME               2 (total)
             18 JUMP_ABSOLUTE            6
        >>   20 POP_BLOCK
        >>   22 LOAD_CONST               0 (None)
             24 RETURN_VALUE


## So what is an iterator?
* a stateful object that produces the next value when you call __`next()`__ on it
* any object that has a __\_\_`next`\_\_()__ method is therefore an iterator
* how it produces a value is irrelevant
* in other words, an iterator is a value factory
 * each time you ask it for "the next" value, it knows how to compute it because it holds internal state

In [14]:
# let's see how an iterator works...
mylist = [13, 46, -3, 'Go!']
myiter = iter(mylist) # get the list iterator

try:
    while True:
        val = next(myiter)
        print(val, end=' ')
except StopIteration:
    print('Stop!')

13 46 -3 Go! Stop!


## Let's build our own iterator!

In [2]:
class Fibonacci(object):
    def __init__(self):
        self.prev = 0
        self.curr = 1

    def __iter__(self):
        return self

    def __next__(self):
        """ 
        each call to next() does two important things:

        1. modify its state for the subsequent next() call
        2. produces a result for the current call
        """
        value = self.curr
        self.curr += self.prev
        self.prev = value
        if value > 1000:
            raise StopIteration
        return value

# Note that this class is both an iterable due to __iter__()
# method and its own iterator, due to __next__() method!

f = Fibonacci()
print(next(f), next(f), 'before the for loop')

for num in f:
   print(num, end=' ')

1 1 before the for loop
2 3 5 8 13 21 34 55 89 144 233 377 610 987 

## Lab: Iterators

Write your own iterator class which takes an iterable and each time it's invoked, it returns a *random* element. The iterator should stop (i.e., __`raise`__ the __`StopIteration`__ exception) when it has returned all elements of the iterable.

Example: __`MyRandomIterator([1, 2, 3])`__ might return

`
2
3
1
...then raise StopIteration`

In [18]:
'''
Note that because we use indexing ([]) below, this iterator
only works for iterables which support indexing.
'''
class MyRandomIterator():
    def __init__(self, iterable):
        self.indicesReturnedSoFar = set()
        self.iterable = iterable
    
    def __iter__(self):
        return self
    
    def __next__(self):
        import random
        if len(self.indicesReturnedSoFar) == len(self.iterable):
            raise StopIteration
        
        rand = random.randint(0, len(self.iterable) - 1)
        while rand in self.indicesReturnedSoFar:
            rand = random.randint(0, len(self.iterable) - 1)
        # better would be to remove the item from our copy of the
        
        self.indicesReturnedSoFar.add(rand)
        return self.iterable[rand]

In [21]:
mri = MyRandomIterator([1, 2, 3, 4])
for item in mri:
    print(item)

2
4
3
1


In [3]:
from collections import namedtuple 

card = namedtuple('Card', ['rank', 'suit'])

class DeckOfCards:
    ranks = list(range(2, 11)) + list('JQKA')
    suits = 'clubs diamonds hearts spades'.split()

    def __init__(self):
        self._cards = [card(rank, suit) for suit in self.suits for rank in self.ranks]
    
    def __str__(self):
        time_to_print = [str(x.rank) + ' of ' + str(x.suit) for x in self._cards]
        return str('\n'.join(time_to_print))
    
    def __iter__(self):
        return iter(self._cards)

deck = DeckOfCards()
for card in deck:
   print(card)
#print(deck)

Card(rank=2, suit='clubs')
Card(rank=3, suit='clubs')
Card(rank=4, suit='clubs')
Card(rank=5, suit='clubs')
Card(rank=6, suit='clubs')
Card(rank=7, suit='clubs')
Card(rank=8, suit='clubs')
Card(rank=9, suit='clubs')
Card(rank=10, suit='clubs')
Card(rank='J', suit='clubs')
Card(rank='Q', suit='clubs')
Card(rank='K', suit='clubs')
Card(rank='A', suit='clubs')
Card(rank=2, suit='diamonds')
Card(rank=3, suit='diamonds')
Card(rank=4, suit='diamonds')
Card(rank=5, suit='diamonds')
Card(rank=6, suit='diamonds')
Card(rank=7, suit='diamonds')
Card(rank=8, suit='diamonds')
Card(rank=9, suit='diamonds')
Card(rank=10, suit='diamonds')
Card(rank='J', suit='diamonds')
Card(rank='Q', suit='diamonds')
Card(rank='K', suit='diamonds')
Card(rank='A', suit='diamonds')
Card(rank=2, suit='hearts')
Card(rank=3, suit='hearts')
Card(rank=4, suit='hearts')
Card(rank=5, suit='hearts')
Card(rank=6, suit='hearts')
Card(rank=7, suit='hearts')
Card(rank=8, suit='hearts')
Card(rank=9, suit='hearts')
Card(rank=10, sui

## Generators
* a generator allows you to write iterators much like the Fibonacci iterator above but in an elegant, succinct syntax that avoids writing classes with __\_\_`iter`\_\_`()`__ and __\_\_`next`\_\_`()`__ methods
* every generator is an iterator (but not vice versa!) 
* a generator is a factory that lazily produces values
* two types: generator _functions_ and generator _expressions_

## The `yield` statement
* before we jump into generators, let's take an in-depth look at what makes them possible...
* when a normal Python function is invoked, execution starts at the first line and continues until a `return` statement is encountered or an exception is thrown (remember that "falling off the end of the function" is the same as if we had written __`return None`__)
    * once a function returns, that's it–any work done by the function and stored in local variables is lost
    * the next call to the function starts everything anew
* there are times when we'd like to have a "function" which yields a series of values, i.e., it would have to save its state to that the next time it's invoked, it picks up where it left off
    * we use the term "yield" here because in fact we are *not returning* to the caller i.e., we are not returning control of execution to the point where the function was called
    * instead of __`return`__-ing, we are __`yield`__-ing, which implies that the transfer of control is temporary and voluntary–our function expects to regain control in the future
* functions that use __`yield`__ instead of __`return`__ are generator functions (or *coroutines* in other languages)
* think of __`yield`__ as __`return`__ + "some magic" for generator functions

### So what's the magic?
* when __`yield`__ is called the state of the generator function is recorded
    * the value of all variables are saved
    * the next line of code to be executed is also saved
    * i.e., the function simply resumes where it left off

In [4]:
def simple_generator():
    yield 1
    yield 'boo!'
    yield 3
    
for value in simple_generator():
    print(value)

1
boo!
3


## Why do we need generator functions?
* initially they gave programmers an easy way to write code that produced a series of values
    * without generator functions, writing something like a random number generator required a class or module that both generated values and kept track of state between calls
    * with generator functions, doing the above is greatly simplified
* suppose we want a function which, given a list of numbers, returns a list of those numbers which are prime
    * straighforward...
    
             def get_primes(nums):
                 return ([num for num in nums if is_prime(num)])

* now suppose we want to use the above function for very large lists of numbers...so large, in fact, that they won't fit in memory
    * so now we want the function to take a starting value, and return all the primes that are greater than that value
    * since functions only return once, they only have one "chance" to return a value (or list of values)
    * what if our function could return the *next* value, rather than a list?
        * we wouldn't need to create a list at all!

## What is a generator function?
* defined like a normal function, but whenever it needs to generate a value, it does so with the __`yield`__ keyword rather than __`return`__
    * if the body of a def contains __`yield`__, the function automatically becomes a generator function (even if it also has a __`return`__ statement)
    * ...there's nothing else we need to do to create one
* generator functions create *generator iterators* (or simply, a *generator*)
    * a generator is a special type of iterator (meaning it has a __\_\_`next`\_\_`()`__ function)
    * to get the next value from a generator, we use the same built-in function as for iterators: __`next()`__
* let's return to the more basic notion of a generator function...

### Now we can rewrite our `get_primes()` function as a generator...

    def get_primes(num):
        while True:
            if is_prime(num):
                yield num
            num += 1
            
* note that if a generator function calls return (or simply hits the end of the function), then a __`StopIteration`__ exception is raised, signaling the generator is exhausted (just as an iterator does)

In [5]:
def fibonacci():
    '''
    defined as a normal function, but...
    ...no return keyword
    
    The yield keyword returns a value, but the function retains its state
    '''
    prev, curr = 0, 1
    while True:
        yield curr
        prev, curr = curr, prev + curr
        
f = fibonacci()
print(next(f), next(f), 'before the for loop', sep='\n')

import random
random.seed()

for num in f:
   print(num, end=' ')
   if random.random() > 0.95:
        break

1
1
before the for loop
2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 10946 17711 28657 46368 75025 121393 196418 317811 514229 832040 1346269 2178309 3524578 5702887 9227465 14930352 24157817 39088169 63245986 102334155 165580141 267914296 433494437 701408733 1134903170 1836311903 2971215073 4807526976 7778742049 12586269025 20365011074 32951280099 53316291173 86267571272 139583862445 225851433717 365435296162 591286729879 956722026041 1548008755920 2504730781961 4052739537881 6557470319842 10610209857723 17167680177565 27777890035288 44945570212853 72723460248141 117669030460994 

In [34]:
fibonacci()

<generator object fibonacci at 0x105f00af0>

## PEP-342: sending values into generators
* PEP-342 added support for passing values *into* generators using the `send()` function
* let's go back to the prime number example but instead of simply printing every prime number greater than some number, we'll find the smallest prime number greater than successive powers of a number (i.e. for 10, we want the smallest prime greater than 10, then 100, then 1000, etc.)

        def get_primes(num):
            while True:
                if is_prime(num):
                    num = yield num 
                num += 1
                
* the `yield` line now says __"yield num, and when a value is sent to me, set num to that value"__
* and we can print the next prime greater than 10, 100, 1000, as follows:

        def print_successive_primes(iterations, base=10):
            prime_generator = get_primes(base)
            prime_generator.send(None)
            for power in range(iterations):
                print(prime_generator.send(base ** power))
                
* printing __`generator.send()`__ is possible because __`send`__ both sends a value to the generator and returns the value yielded by the generator
* note that the first time we send a value into a generator, it must be __`None`__

## Now let's look at a generator _expression_
* generator equivalent of a list comprehension

In [41]:
squares = [num * num for num in range(1, 11)] # list comprehension
squares

[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

In [42]:
squares = {num * num for num in range(1, 11)} # set comprehension
squares

{1, 4, 9, 16, 25, 36, 49, 64, 81, 100}

In [58]:
squares = {num: num * num for num in range(1, 11)} # dict comprehension
squares

{1: 1, 2: 4, 3: 9, 4: 16, 5: 25, 6: 36, 7: 49, 8: 64, 9: 81, 10: 100}

In [1]:
squares = (num * num for num in range(1, 11)) # generator expression (NOT a 'tuple comprehension')
squares

<generator object <genexpr> at 0x1052b6360>

In [2]:
next(squares), next(squares)

(1, 4)

In [7]:
iter(range(1, 10))

In [3]:
list(squares) # for thing in squares: print(thing)

[9, 16, 25, 36, 49, 64, 81, 100]

## Lab: Generators
* modify your random iterator to be a generator function

In [55]:
def my_random_generator(iterable):
    import random
    indices_returned_so_far = set()
    
    while len(indices_returned_so_far) < len(iterable):
        rand = random.randint(0, len(iterable) - 1)
        while rand in indices_returned_so_far:
            rand = random.randint(0, len(iterable) - 1)

        indices_returned_so_far.add(rand)
        yield iterable[rand]
        
for thing in my_random_generator([1, 2, 3, 4]):
    print(thing)

2
1
3
4


## __`itertools`__
* module of functions for efficient looping
* all of its methods return iterators
* some produce finite sequences
* others produce infinite sequences

In [56]:
from itertools import zip_longest
list1 = ['a', 'b', 'c', 'd']
list2 = ['apple', 'banana', 'cherry']

for item1, item2 in zip_longest(list1, list2):
    print(item1, '=>', item2)

a => apple
b => banana
c => cherry
d => None


In [8]:
from itertools import count
counter = count(start=789)
next(counter), next(counter)

(789, 790)

In [10]:
from itertools import count

for num in count(1, 0.25):
    print(num)
    if num > 7.75:
        break

1
1.25
1.5
1.75
2.0
2.25
2.5
2.75
3.0
3.25
3.5
3.75
4.0
4.25
4.5
4.75
5.0
5.25
5.5
5.75
6.0
6.25
6.5
6.75
7.0
7.25
7.5
7.75
8.0


In [8]:
from itertools import cycle
sizes = ['S', 'M', 'L']
sc = cycle(sizes)
next(sc), next(sc), next(sc), next(sc), next(sc)

('S', 'M', 'L', 'S', 'M')

In [6]:
from itertools import islice
list(islice(fibonacci(), 56, 61))

[365435296162, 591286729879, 956722026041, 1548008755920, 2504730781961]

In [64]:
from itertools import count, islice

for num in islice(count(1, 0.25), 1, 5):
    print(num)

1.25
1.5
1.75
2.0


In [67]:
# some produce a finite sequence from an infinite sequence
from itertools import islice, cycle
colors = cycle(['red', 'white', 'blue']) # infinite
limited = islice(colors, 0, 5)

for color in limited:
    print(color, end= ' ')

red white blue red white 

In [68]:
from itertools import chain
rank = list(range(2, 11))
picture = { 'J': 'Jack', 'Q': 'Queen', 'K': 'King', 'A': 'Ace' }

for card in chain(rank, picture):
    print(card, end=' ')
    
#list(chain(rank, picture))

[2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A']

In [20]:
from itertools import filterfalse
# filter out items for which predicate is False
numbers = [7, 12, 20, 23, 32, 44]
list(filterfalse(lambda x: x % 2, numbers))

[12, 20, 32, 44]

In [68]:
# filters elements, returning only those that have a corresponding
# element that evaluates to True
from itertools import compress
words = ['how', 'now', 'brown', 'cow']
counts = [13, '', 'x', None]
list(compress(words, counts))

['how', 'brown']

In [71]:
# accumulate sums, or other binary functions
from itertools import accumulate
list(accumulate([3, 5, 10, 21]))
#help(accumulate)

[3, 8, 18, 39]

In [3]:
list(accumulate(range(1, 10), lambda x, y: x * y))

[1, 2, 6, 24, 120, 720, 5040, 40320, 362880]

In [5]:
# Amortize a 5% loan of 1000 with 4 annual payments of 250
cashflows = [1000, 250, 250, 250, 250]
list(accumulate(cashflows, lambda bal, pmt: bal * 1.05 - pmt))

[1000, 800.0, 590.0, 369.5, 137.97500000000002]